<a href="https://colab.research.google.com/github/secutron/ShowTime/blob/master/tmpPatients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!nvidia-smi

Thu Jun 18 06:22:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    78W / 149W |     74MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
!apt-get install default-jre
!java -version

!pip install h2o

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


In [25]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [26]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,32 mins 43 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_unknownUser_4bzzcl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.178 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [27]:
data = h2o.import_file('column_2C_weka.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
data.head()

pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
63.0278,22.5526,39.6091,40.4752,98.6729,-0.2544,Abnormal
39.057,10.061,25.0154,28.996,114.405,4.56426,Abnormal
68.832,22.2185,50.0922,46.6135,105.985,-3.53032,Abnormal
69.297,24.6529,44.3112,44.6441,101.868,11.2115,Abnormal
49.7129,9.65207,28.3174,40.0608,108.169,7.9185,Abnormal
40.2502,13.9219,25.1249,26.3283,130.328,2.23065,Abnormal
53.4329,15.8643,37.1659,37.5686,120.568,5.98855,Abnormal
45.3668,10.7556,29.0383,34.6111,117.27,-10.6759,Abnormal
43.7902,13.5338,42.6908,30.2564,125.003,13.289,Abnormal
36.6864,5.01088,41.9488,31.6755,84.2414,0.664437,Abnormal


In [29]:
data.types

{'class': 'enum',
 'degree_spondylolisthesis': 'real',
 'lumbar_lordosis_angle': 'real',
 'pelvic_incidence': 'real',
 'pelvic_radius': 'real',
 'pelvic_tilt numeric': 'real',
 'sacral_slope': 'real'}

In [30]:
data.describe()   

Rows:310
Cols:7




,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
type,real,real,real,real,real,real,enum
mins,26.14792141,-6.554948347,14.0,13.3669307,70.08257486,-11.05817866,
mean,60.496652929516145,17.54282196797096,51.93092960345161,42.95383096141935,117.92065502380645,26.296694437867743,
maxs,129.8340406,49.4318636,125.7423855,121.4295656,163.0710405,418.5430821,
sigma,17.236520321708866,10.008330258206355,18.55406396276117,13.423102164839573,13.31737704490458,37.55902655487235,
zeros,0,0,0,0,0,0,
missing,0,0,0,0,0,0,0
0,63.0278175,22.55258597,39.60911701,40.47523153,98.67291675,-0.254399986,Abnormal
1,39.05695098,10.06099147,25.01537822,28.99595951,114.4054254,4.564258645,Abnormal
2,68.83202098,22.21848205,50.09219357,46.61353893,105.9851355,-3.530317314,Abnormal


In [31]:
#Train:- 70%
# Test:- 15%
# Validation :- 15%

In [32]:
data_train,data_test,data_valid = data.split_frame(ratios=[.7, .15])

In [33]:
data_train

pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
63.0278,22.5526,39.6091,40.4752,98.6729,-0.2544,Abnormal
39.057,10.061,25.0154,28.996,114.405,4.56426,Abnormal
69.297,24.6529,44.3112,44.6441,101.868,11.2115,Abnormal
49.7129,9.65207,28.3174,40.0608,108.169,7.9185,Abnormal
40.2502,13.9219,25.1249,26.3283,130.328,2.23065,Abnormal
43.7902,13.5338,42.6908,30.2564,125.003,13.289,Abnormal
36.6864,5.01088,41.9488,31.6755,84.2414,0.664437,Abnormal
49.7066,13.041,31.3345,36.6656,108.648,-7.82599,Abnormal
48.9156,19.9646,40.2638,28.951,119.321,8.02889,Abnormal
44.3189,12.538,36.0988,31.7809,124.116,5.41583,Abnormal


In [34]:
data_test

pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
68.832,22.2185,50.0922,46.6135,105.985,-3.53032,Abnormal
53.4329,15.8643,37.1659,37.5686,120.568,5.98855,Abnormal
45.3668,10.7556,29.0383,34.6111,117.27,-10.6759,Abnormal
31.2324,17.7158,15.5,13.5166,120.055,0.499751,Abnormal
57.3002,24.1889,47,33.1113,116.807,5.76695,Abnormal
54.9194,21.0623,42.2,33.8571,125.213,2.43256,Abnormal
32.091,6.98938,35.9982,25.1016,132.265,6.41343,Abnormal
55.8433,28.8474,47.6905,26.9958,123.312,2.81243,Abnormal
53.8548,19.2306,32.7791,34.6242,121.671,5.32984,Abnormal
48.3326,22.2278,36.182,26.1049,117.385,6.48171,Abnormal


In [35]:
data_valid

pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
53.5722,20.4608,33.1,33.1113,110.967,7.0448,Abnormal
54.1249,26.6505,35.3297,27.4744,121.447,1.5712,Abnormal
50.8193,15.4022,42.5289,35.4171,112.193,10.8696,Abnormal
46.3903,11.079,32.1366,35.3112,98.7745,6.38683,Abnormal
35.4924,11.7017,15.5904,23.7908,106.939,-3.46036,Abnormal
66.2854,26.3278,47.5,39.9575,121.22,-0.799624,Abnormal
56.0302,16.2979,62.2753,39.7323,114.023,-2.32568,Abnormal
41.7677,17.8994,20.0309,23.8683,118.363,2.06296,Abnormal
41.1717,17.3212,33.4694,23.8505,116.378,-9.56925,Abnormal
74.3777,32.0531,78.772,42.3246,143.561,56.1259,Abnormal


### 학습용 데이터 구성

In [36]:
y = "class"
x = data.columns
x.remove(y)

In [38]:
# aml 생성
aml = H2OAutoML(max_models = 10, seed = 10, exclude_algos = ["StackedEnsemble"], verbosity="info", nfolds=0)

In [39]:
!nvidia-smi    

Thu Jun 18 06:23:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    78W / 149W |     74MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [40]:
# 모델 트레이닝
aml.train(x = x, y = y, training_frame = data_train, validation_frame=data_valid)

AutoML progress: |
06:23:39.289: Project: AutoML_20200618_62339288
06:23:39.289: Cross-validation disabled by user: no fold column nor nfolds > 1.
06:23:39.289: Setting stopping tolerance adaptively based on the training frame: 0.05
06:23:39.289: Build control seed: 10
06:23:39.290: Since cross-validation is disabled, and no leaderboard frame was provided, automatically split the training data into training and leaderboard frames in the ratio 90/10
06:23:39.297: training frame: Frame key: automl_training_py_53_sid_b07b    cols: 7    rows: 189  chunks: 1    size: 10146  checksum: 2890451132143199720
06:23:39.297: validation frame: Frame key: py_55_sid_b07b    cols: 7    rows: 53  chunks: 1    size: 3601  checksum: -901609713707294612
06:23:39.301: leaderboard frame: Frame key: automl_leaderboard_py_53_sid_b07b    cols: 7    rows: 25  chunks: 1    size: 2254  checksum: 3814782166834657900
06:23:39.301: blending frame: NULL
06:23:39.301: response column: class
06:23:39.301: fold column: n

In [41]:
!nvidia-smi

Thu Jun 18 06:23:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    78W / 149W |     74MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [42]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GLM_1_AutoML_20200618_062339,0.984127,0.241929,0.958903,0.0277778,0.26753,0.0715725
DRF_1_AutoML_20200618_062339,0.920635,0.345768,0.843328,0.154762,0.343424,0.11794
GBM_4_AutoML_20200618_062339,0.912698,0.340621,0.82299,0.170635,0.3269,0.106864
GBM_2_AutoML_20200618_062339,0.904762,0.414893,0.768313,0.111111,0.373265,0.139327
XGBoost_1_AutoML_20200618_062339,0.888889,0.405603,0.754168,0.198413,0.371068,0.137691
DeepLearning_1_AutoML_20200618_062339,0.888889,0.620921,0.825265,0.170635,0.457309,0.209132
XGBoost_3_AutoML_20200618_062339,0.880952,0.423932,0.746295,0.198413,0.374454,0.140216
GBM_1_AutoML_20200618_062339,0.873016,0.482805,0.751789,0.18254,0.40099,0.160793
GBM_3_AutoML_20200618_062339,0.865079,0.431799,0.582623,0.138889,0.377808,0.142739
XGBoost_2_AutoML_20200618_062339,0.857143,0.427635,0.625343,0.198413,0.37246,0.138726


In [44]:
# prediction
data_pred=aml.leader.predict(data_test)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [45]:
data_pred.head()

predict,Abnormal,Normal
Abnormal,0.38487,0.61513
Abnormal,0.507777,0.492223
Normal,0.131623,0.868377
Abnormal,0.812599,0.187401
Abnormal,0.750482,0.249518
Abnormal,0.44805,0.55195
Abnormal,0.42557,0.57443
Abnormal,0.724797,0.275203
Abnormal,0.573245,0.426755
Abnormal,0.836323,0.163677


### 성능 측정

In [46]:
aml.leader.model_performance(data_test)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.10873856412251652
RMSE: 0.32975530946827303
LogLoss: 0.33738225333810323
Null degrees of freedom: 42
Residual degrees of freedom: 36
Null deviance: 55.81041178421261
Residual deviance: 29.014873787076883
AIC: 43.01487378707688
AUC: 0.9238095238095239
AUCPR: 0.8478419068957422
Gini: 0.8476190476190477

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6069064137325721: 


,,Abnormal,Normal,Error,Rate
0,Abnormal,26.0,2.0,0.0714,(2.0/28.0)
1,Normal,3.0,12.0,0.2,(3.0/15.0)
2,Total,29.0,14.0,0.1163,(5.0/43.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,6.069064e-01,0.827586,13.0
1,max f2,2.137970e-01,0.882353,24.0
2,max f0point5,6.421968e-01,0.873016,11.0
3,max accuracy,6.421968e-01,0.883721,11.0
4,max precision,9.450152e-01,1.000000,0.0
5,max recall,2.137970e-01,1.000000,24.0
6,max specificity,9.450152e-01,1.000000,0.0
7,max absolute_mcc,6.421968e-01,0.741262,11.0
8,max min_per_class_accuracy,5.168070e-01,0.821429,17.0
9,max mean_per_class_accuracy,6.069064e-01,0.864286,13.0



Gains/Lift Table: Avg response rate: 34.88 %, avg score: 37.75 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.023256,9.313621e-01,2.866667,2.866667,1.00,0.945015,1.000000,0.945015,0.066667,0.066667,186.666667,186.666667
1,,2,0.023256,9.177090e-01,0.000000,2.866667,0.00,0.000000,1.000000,0.945015,0.000000,0.066667,-100.000000,186.666667
2,,3,0.046512,9.048002e-01,2.866667,2.866667,1.00,0.912508,1.000000,0.928762,0.066667,0.133333,186.666667,186.666667
3,,4,0.046512,8.923493e-01,0.000000,2.866667,0.00,0.000000,1.000000,0.928762,0.000000,0.133333,-100.000000,186.666667
4,,5,0.069767,8.814144e-01,2.866667,2.866667,1.00,0.882863,1.000000,0.913462,0.066667,0.200000,186.666667,186.666667
5,,6,0.116279,8.297799e-01,1.433333,2.293333,0.50,0.849501,0.800000,0.887878,0.066667,0.266667,43.333333,129.333333
6,,7,0.162791,8.032582e-01,2.866667,2.457143,1.00,0.819848,0.857143,0.868441,0.133333,0.400000,186.666667,145.714286
7,,8,0.209302,7.660151e-01,2.866667,2.548148,1.00,0.776188,0.888889,0.847940,0.133333,0.533333,186.666667,154.814815
8,,9,0.302326,6.101960e-01,2.150000,2.425641,0.75,0.682436,0.846154,0.797016,0.200000,0.733333,115.000000,142.564103
9,,10,0.395349,5.238355e-01,0.716667,2.023529,0.25,0.575360,0.705882,0.744862,0.066667,0.800000,-28.333333,102.352941
